# Understand the kernel with code samples

----
The kernel is the central component of Semantic Kernel. At its simplest, the kernel is a Dependency Injection container that manages all of the services and plugins necessary to run your AI application. If you provide all of your services and plugins to the kernel, they will then be seamlessly used by the AI as needed. 

| Components | Description |
|------------|-------------|
| **1. Services** | These consist of both AI services (e.g., chat completion) and other services (e.g., logging and HTTP clients) that are necessary to run your application. This was modeled after the Service Provider pattern in .NET to support dependency injection across all languages. |
| **2. Plugins** | These are components used by your AI services and prompt templates to perform work. For example, AI services can use plugins to retrieve data from a database or call an external API to perform actions. |

![kernel](images/the-kernel-is-at-the-center-of-everything.png)



## Hierarchical Architecture Diagram
---

Architecture of a Retrieval-Augmented Generation (RAG) based chatbot using the latest Semantic Kernel. The diagram highlights how the Agent Framework (for chat interactions) and Process Framework (for workflow orchestration) sit above the core Kernel, leveraging it to perform AI tasks. The Semantic Kernel core orchestrates between user requests and the underlying AI services, memory stores, and plugins. It integrates with external LLM services for generation, uses a vector DB for semantic memory (retrieval of relevant data), and can invoke plugins which may call other external APIs or services as needed.

![Hierarchical](images/semantic_kernel_component.png)

***AI Agent (Agent Framework)***: The Agent Framework in Semantic Kernel is an optional layer that helps create conversational AI agents (like chatbots) using the core kernel’s capabilities​. It is not a replacement for the kernel but builds on top of it – your application still includes the Semantic Kernel library, and the agent uses the kernel’s functions internally. In a chatbot scenario, the Agent Framework manages the dialogue (turn-taking, system prompts, etc.) while delegating AI tasks to the kernel.

***Semantic Kernel (Core)***: The Semantic Kernel core is the heart of the system. It orchestrates calls to AI models, retrieves memories, and executes plugin functions. The kernel provides abstractions to connect to AI services (LLM APIs for text generation, embedding models, etc.) and to various memory stores (for example, vector databases)​. It also manages context (prompts) and can use Planners to chain or select functions dynamically to fulfill a user request​. The kernel itself is part of your app’s runtime, coordinating all other components.

***Plugins (Skills/Functions)***: Plugins (also called skills or functions) are units of functionality that the kernel can invoke. They might be defined with natural language prompts (semantic functions) or as native code functions. Plugins can perform calculations, transform data, or call external services/APIs. They are registered with and executed via the kernel, meaning they depend on the kernel to be invoked as part of an AI workflow​. However, the plugin implementations (e.g. an HTTP call to a web service, a database query) run outside the kernel – the kernel just orchestrates their usage. In essence, plugins extend the kernel’s abilities, and the kernel can automatically chain plugins to accomplish complex tasks for the user​.

***AI Services***: These are external AI model endpoints that the kernel calls through its connectors. For example, the OpenAI or Azure OpenAI service provides the GPT-4 model for text generation, and there are embedding model services for vector generation. The kernel has integrations for many AI services (text completion, chat, image generation, speech recognition, etc.) and it uses them by calling out to the respective APIs​. These services are not “inside” the kernel – instead, the kernel depends on them to provide the intelligence. In the architecture, they appear as external components that the kernel invokes (e.g. sending a prompt to the GPT model and getting a completion).

***Semantic Memory (Vector DB)***: The Semantic Kernel includes a memory abstraction that allows storing and retrieving contextual information. Under the hood, this is often backed by a vector database or search index. In a RAG-based chatbot, this is critical: documents or knowledge are embedded into vector representations and stored, so that relevant pieces can be retrieved to ground the AI’s answers. Semantic Kernel’s memory can integrate with many vector stores (e.g. Azure Cognitive Search, ChromaDB, Qdrant, Redis, Pinecone, etc.)​. This means “Memory” is essentially an AI Search over a vector DB, enabling the bot to find relevant information by semantic similarity. The memory component is used by the kernel but the database itself is external – the kernel just sends queries and gets results.

***Process Framework (Workflow Orchestration)***: The Process Framework is another optional layer in the latest Semantic Kernel, aimed at long-running or multi-step workflows. It lets developers define structured processes composed of multiple steps, where each step can call kernel functions (AI or non-AI tasks) in an event-driven sequence​. Like the Agent Framework, the Process Framework builds on the kernel rather than enclosing it. It uses the kernel to execute AI functions at each step of a business workflow. This is especially useful if your chatbot or assistant needs to carry out complex transactions or procedures (for example, an order processing workflow or a support ticket resolution that involves several back-and-forth steps) beyond a single conversational turn. The Process Framework uses technologies like Orleans or Dapr under the hood for reliability and can reuse any existing kernel plugins within those processes​.

# Setup

Let's define our kernel for this example.

In [31]:
import asyncio
from typing import Annotated
from dotenv import load_dotenv
import json
import os
from enum import Enum
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.agents import AssistantAgentThread, AzureAssistantAgent
from semantic_kernel.kernel_pydantic import KernelBaseSettings
from semantic_kernel.connectors.search.bing import BingSearch
from semantic_kernel.functions import KernelArguments, KernelParameterMetadata, KernelPlugin
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.contents import ChatHistory
from semantic_kernel.filters import FilterTypes, FunctionInvocationContext
from collections.abc import Awaitable, Callable
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.agents import AzureResponsesAgent, ResponsesAgentThread
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents import AzureResponsesAgent
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents import AuthorRole
from semantic_kernel.contents.streaming_chat_message_content import StreamingChatMessageContent
from azure.ai.agents.models import OpenApiAnonymousAuthDetails, OpenApiTool, CodeInterpreterTool

kernel = Kernel()



class Service(Enum):
    """Attributes:
    OpenAI (str): Represents the OpenAI service.
    AzureOpenAI (str): Represents the Azure OpenAI service.
    HuggingFace (str): Represents the HuggingFace service.
    """

    OpenAI = "openai"
    AzureOpenAI = "azureopenai"
    HuggingFace = "huggingface"

class ServiceSettings(KernelBaseSettings):
    """The Learn Resources Service Settings.

    The settings are first loaded from environment variables. If the
    environment variables are not found, the settings can be loaded from a .env file with the
    encoding 'utf-8' as default or the specific encoding. If the settings are not found in the
    .env file, the settings are ignored; however, validation will fail alerting that the settings
    are missing.

    Args:
        global_llm_service (str | None): The LLM service to use for the samples, either "OpenAI" or "AzureOpenAI"
            If not provided, defaults to "AzureOpenAI".
    """

    global_llm_service: str | None = None
    
load_dotenv(override=True)

True

We will load our settings and get the LLM service to use for the notebook.

In [2]:
service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

The create method is deprecated. Use the __new__ method instead.


Using service type: Service.AzureOpenAI


We now configure our Chat Completion service on the kernel.

In [3]:
# Remove all services so that this cell can be re-run without restarting the kernel
kernel.remove_all_services()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

# Run a Semantic Function (No Agent, No Process)


## 🧪 Case 1.1 running a prompt without plugin
---

In [4]:


# Define the request settings
req_settings = kernel.get_prompt_execution_settings_from_service_id(service_id)
req_settings.max_tokens = 2000
req_settings.temperature = 0.7
req_settings.top_p = 0.8

prompt="tell me about the history of {{$company_name}} as TLDR in exactly 200 words. make line seperated by commas. \n\nTLDR:"

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="tldr",
    template_format="semantic-kernel",
    execution_settings=req_settings,
)

function = kernel.add_function(
    function_name="tldr_function",
    plugin_name="tldr_plugin",
    prompt_template_config=prompt_template_config,
)


########################################
# invoke mode 
########################################

# result = await kernel.invoke(function)
# print(result) # => Robots must not harm humans.

########################################
# invoke_stream mode 
########################################

response = kernel.invoke_stream(plugin_name="tldr_plugin", function_name="tldr_function", company_name="Microsoft")
all_chunks_str = ""
print("Assistant:> ", end="")
async for chunk in response:
    if isinstance(chunk[0], StreamingChatMessageContent) and chunk[0].role == AuthorRole.ASSISTANT:
        all_chunks_str += str(chunk[0])
        print(str(chunk[0]), end="")
            
            

Assistant:> 

Microsoft was founded by Bill Gates and Paul Allen in 1975,  
Initially created a version of the BASIC programming language for Altair 8800,  
In 1980, they signed a contract with IBM to provide an operating system,  
This led to the creation of MS-DOS, which became a dominant OS for PCs,  
In 1985, Microsoft launched Windows 1.0, a graphical extension for MS-DOS,  
Windows 3.0 was released in 1990, gaining significant market share,  
The 1995 launch of Windows 95 marked a major success, featuring a new user interface,  
Microsoft Office suite was introduced, becoming essential for businesses,  
The late 1990s saw antitrust lawsuits against Microsoft for monopolistic practices,  
In 2001, Windows XP was released, praised for its stability and user-friendly design,  
The rise of the internet led to the launch of Internet Explorer, competing with Netscape,  
In 2007, Microsoft introduced Windows Vista, followed by Windows 7 in 2009,  
The company expanded into gaming with the Xbox, launc

## 🧪 Case 1.2 running a prompt with plugin
---

In [5]:
plugin = kernel.add_plugin(parent_directory="prompt_template_samples/", plugin_name="FunPlugin")

In [6]:
from semantic_kernel.functions import KernelArguments

joke_function = plugin["Joke"]

joke = await kernel.invoke(
    joke_function,
    KernelArguments(input="time travel to dinosaur age", style="super silly"),
)
print(joke)

Why did the time traveler bring a ladder to the dinosaur age?

Because he heard the T-Rex was a little "short" on friends and wanted to help him reach new heights in socializing! 🦖😂


## 🧪 Case 1.3 running a prompt with Bing Search v7 plugin
---
- This case shows how to use the Bing Search v7 plugin to search the web and return results.
- You must have a valid Bing Search v7 API key to run this case.

In [27]:
import os

import semantic_kernel.connectors.search.bing

# BING_API_KEY needs to be set in your environment variables
if not os.getenv("BING_API_KEY"):
    raise ValueError("BING_API_KEY environment variable is not set. Please set it to your Bing Search API key.")


kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id="chat"))
bing_search_plugin = kernel.add_plugin(KernelPlugin.from_text_search_with_search(
        semantic_kernel.connectors.search.bing.BingSearch(),
        plugin_name="bing",
        description="Search the web information using bing search.",
        parameters=[
            KernelParameterMetadata(
                name="query",
                description="The search query.",
                type="str",
                is_required=True,
                type_object=str,
            ),
            KernelParameterMetadata(
                name="top",
                description="The number of results to return.",
                type="int",
                is_required=False,
                default_value=2,
                type_object=int,
            ),
            KernelParameterMetadata(
                name="skip",
                description="The number of results to skip.",
                type="int",
                is_required=False,
                default_value=0,
                type_object=int,
            ),
            KernelParameterMetadata(
                name="site",
                description="The site to search.",
                default_value="",
                type="str",
                is_required=False,
                type_object=str,
            ),
        ],
    ))

chat_function = kernel.add_function(
    prompt="{{$chat_history}}{{$user_input}}",
    plugin_name="ChatBot",
    function_name="Chat",
)
execution_settings = AzureChatPromptExecutionSettings(
    service_id="chat",
    max_tokens=2000,
    temperature=0.7,
    top_p=0.8,
    function_choice_behavior=FunctionChoiceBehavior.Auto(auto_invoke=True),
)

history = ChatHistory()
system_message = """
You are a chat bot, use bing plugin to find answers.
"""
history.add_system_message(system_message)
history.add_user_message("Hi there, who are you?")
history.add_assistant_message("I am Information Finder, a chat bot. use bing plugin to find answers.")

arguments = KernelArguments(settings=execution_settings)

@kernel.filter(filter_type=FilterTypes.FUNCTION_INVOCATION)
async def log_bing_filter(
    context: FunctionInvocationContext, next: Callable[[FunctionInvocationContext], Awaitable[None]]
):
    if context.function.plugin_name == "bing":
        print("Calling Bing search with arguments:")
        if "query" in context.arguments:
            print(f'  Query: "{context.arguments["query"]}"')
        if "count" in context.arguments:
            print(f'  Count: "{context.arguments["count"]}"')
        if "skip" in context.arguments:
            print(f'  Skip: "{context.arguments["skip"]}"')
        await next(context)
        print("Bing search completed.")
    else:
        await next(context)


async def chat() -> bool:
    try:
        user_input = input("User:> ")
    except KeyboardInterrupt:
        print("\n\nExiting chat...")
        return False
    except EOFError:
        print("\n\nExiting chat...")
        return False

    if user_input == "exit":
        print("\n\nExiting chat...")
        return False
    arguments["user_input"] = user_input
    arguments["chat_history"] = history
    result = await kernel.invoke(chat_function, arguments=arguments)
    print(f"SearchChatbot:> {result}")
    history.add_user_message(user_input)
    history.add_assistant_message(str(result))
    return True



chatting = True
print(
    "Welcome to the chat bot!\
    \n  Type 'exit' to exit.\
    \n  Try to search weather, news and more using bing search tool."
)
while chatting:
    chatting = await chat()

Welcome to the chat bot!    
  Type 'exit' to exit.    
  Try to search weather, news and more using bing search tool.


Calling Bing search with arguments:
  Query: "가장 좋은 인공지능 모델 2023"
Bing search completed.
SearchChatbot:> 2023년 가장 좋은 인공지능 모델로는 다음과 같은 모델들이 있습니다:

1. **Falcon LLM**: 이 모델은 1,800억 개의 매개변수를 가지고 있으며, 3조 5,000억 개의 토큰으로 학습되었습니다. 허깅페이스의 오픈 LLM 리더보드에서 1위를 차지했으며, 상업용 및 연구용으로 모두 사용 가능합니다.

2. **Gemini**: 구글과 딥마인드가 개발한 멀티모달 생성형 인공지능 모델로, 2023년 5월에 처음 공개되었습니다. 다양한 기능을 제공하며, 여러 가지 형태의 데이터를 처리할 수 있습니다.

3. **Adobe Firefly**: Adobe의 Firefly는 일러스트레이션, 아트 컨셉, 사진 조작 등 다양한 시각 콘텐츠 생성을 가능하게 하며, Photoshop에 통합되어 사용자에게 AI의 힘을 제공합니다.

이 외에도 여러 인공지능 모델들이 존재하지만, 위의 모델들이 특히 주목받고 있습니다.


Exiting chat...


In [8]:
from semantic_kernel.functions import KernelArguments

bing_search_fuction = kernel.get_function("bing", "search")

search_result = await kernel.invoke(
    bing_search_fuction,
    KernelArguments(
        query="what is new opening hotels in NYC 2025?",
        top=10,
        skip=0,
    ),
)
print(search_result)

Calling Bing search with arguments:
  Query: "what is new opening hotels in NYC 2025?"
  Skip: "0"


Bing search completed.
We try to keep our pages about new hotels in NYC as up-to-date as possible and will be updating this page when any new hotels open or when any upcoming hotels in NYC open for reservation.,New York is poised to welcome new hotels in New York City and beyond in 2025. Add these five to your must-stay list.,Now Now NoHo – A Unique Sleeper Cabin Hotel in Manhattan Now Now NoHo is a distinctive sleeper cabin hotel set to open on April 1, 2025, at 338 Bowery in Manhattan’s NoHo neighborhood.,Opened Hotels in New York We have created a list with the best newly opened hotels in New York, USA. Hope you enjoy it. We think it might be the best of its kind around. Tip: Set your dates to see room prices!,A list of my most anticipated New Hotel Openings or Renovations for 2025. Most, but not all, are Resorts, many are Luxury or Boutique properties. Hopefully they will be bringing something new and unique to the world of hospitality.,The top luxury hotel openings in 2025, bringi

# Use single agent to chat


In [ ]:
###################################################################
# The following sample demonstrates how to create a simple,       #
# Azure AI agent, ChatCompletionAgent, AzureResponsesAgent        #
# that answers questions about a sample menu                      #
# using a Semantic Kernel Plugin.                                 #
###################################################################


# Define a sample plugin for the sample
class MenuPlugin:
    """A sample Menu Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of specials from the menu.")
    def get_specials(self) -> Annotated[str, "Returns the specials from the menu."]:
        return """
        Special Soup: Clam Chowder
        Special Salad: Cobb Salad
        Special Drink: Chai Tea
        """

    @kernel_function(description="Provides the price of the requested menu item.")
    def get_item_price(
        self, menu_item: Annotated[str, "The name of the menu item."]
    ) -> Annotated[str, "Returns the price of the menu item."]:
        return "$9.99"


In [16]:
USER_INPUTS = [
    "Hello",
    "What is the special soup?",
    "How much does that cost?",
    "Thank you",
]

## 🧪 Case 2.1 chat with Azure AI Agent 
---

In [43]:
# Azure AI Agent Endpoint and model deployment name need to be set in your environment variables
if not os.getenv("AZURE_AI_AGENT_ENDPOINT"):
    raise ValueError("AZURE_AI_AGENT_ENDPOINT environment variable is not set. Please set it to your Azure AI Agent endpoint.")
if not os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME"):
    raise ValueError("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME environment variable is not set. Please set it to your Azure AI Agent model deployment name.")

# Make sure to set the environment variables for the Azure AI Agent
ai_agent_settings = AzureAIAgentSettings()


async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(credential=creds) as client,
):
    # 1. Create an agent on the Azure AI agent service
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name="Host",
        instructions="Answer questions about the menu.",
    )

    # 2. Create a Semantic Kernel agent for the Azure AI agent
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[MenuPlugin()],  # Add the plugin to the agent
    )

    # 3. Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread = None

########################################
# invoke mode 
########################################

    # for user_input in USER_INPUTS:
    #     print(f"# User: {user_input}")
    #     # 4. Invoke the agent for the specified thread for response
    #     async for response in agent.invoke(
    #         messages=user_input,
    #         thread=thread,
    #     ):
    #         print(f"# {response.name}: {response}")
    #         thread = response.thread
    
    # await thread.delete() if thread else None    

########################################
# invoke_stream mode 
########################################
# Simulate a conversation with the agent

    try:
        for user_input in USER_INPUTS:
            print(f"# User: '{user_input}'")
            first_chunk = True
            # 4. Invoke the agent for the current message and print the response
            async for response in agent.invoke_stream(messages=user_input, thread=thread):
                thread = response.thread
                if first_chunk:
                    print(f"# {response.name}: ", end="", flush=True)
                    first_chunk = False
                print(response.content, end="", flush=True)
            print()
    finally:
            # Cleanup: Delete the thread and agent
            await thread.delete() if thread else None
            await client.agents.delete_agent(agent.id)
        

# User: 'What is the name and population of the country that uses currency with abbreviation KRW'
# Host: The country that uses the currency with the abbreviation **KRW** (Korean Won) is **South Korea**. As of 2023, the population of South Korea is approximately **52 million** people.
# User: 'What is the current weather in the capital city of the country?'
# Host: I cannot provide real-time weather updates. To find the current weather in **Seoul**, the capital city of South Korea, you can check a weather service like [Weather.com](https://www.weather.com) or use a weather app for the most accurate and up-to-date information.


## 🧪 Case 2.2 chat with ChatCompletionAgent
---

In [34]:
#  need to be set in your environment variables - from semantic_kernel.connectors.ai.open_ai.settings.azure_open_ai_settings import AzureOpenAISettings
if not os.getenv("AZURE_OPENAI_ENDPOINT"):
    raise ValueError("AZURE_OPENAI_ENDPOINT environment variable is not set. Please set it to your Azure OpenAI endpoint.")
if not os.getenv("AZURE_OPENAI_API_KEY"):
    raise ValueError("AZURE_OPENAI_API_KEY environment variable is not set. Please set it to your Azure OpenAI API key.")
if not os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"):
    raise ValueError("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME environment variable is not set. Please set it to your Azure OpenAI chat deployment name.")



agent = ChatCompletionAgent(
        service=AzureChatCompletion(),
        name="Host",
        instructions="Answer questions based on web search",
        plugins=[MenuPlugin()],
    )

# 2. Create a thread to hold the conversation
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: ChatHistoryAgentThread = None

########################################
# invoke mode 
########################################

# for user_input in USER_INPUTS:
#     print(f"# User: {user_input}")
#     # 4. Invoke the agent for a response
#     response = await agent.get_response(messages=user_input, thread=thread)
#     print(f"# {response.name}: {response} ")
#     thread = response.thread

########################################
# invoke_stream mode 
########################################

for user_input in USER_INPUTS:
        print(f"# User: '{user_input}'")
        first_chunk = True
        # 4. Invoke the agent for the current message and print the response
        async for response in agent.invoke_stream(messages=user_input, thread=thread):
            thread = response.thread
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(response.content, end="", flush=True)
        print()


await thread.delete() if thread else None

# User: 'Hello'
# Host: 

Hello! How can I assist you today?
# User: 'What is the special soup?'
# Host: The special soup is Clam Chowder. Would you like to know more about it or anything else?
# User: 'How much does that cost?'
# Host: The Clam Chowder costs $9.99. Would you like to know anything else?
# User: 'Thank you'
# Host: You're welcome! If you have any more questions or need assistance, feel free to ask. Have a great day!


## 🧪 Case 2.3 chat with ResponseAgent (OpenAI Responses API)
---

In [30]:
#  need to be set in your environment variables - from semantic_kernel.connectors.ai.open_ai.settings.azure_open_ai_settings import AzureOpenAISettings
if not os.getenv("AZURE_OPENAI_ENDPOINT"):
    raise ValueError("AZURE_OPENAI_ENDPOINT environment variable is not set. Please set it to your Azure OpenAI endpoint.")
if not os.getenv("AZURE_OPENAI_API_KEY"):
    raise ValueError("AZURE_OPENAI_API_KEY environment variable is not set. Please set it to your Azure OpenAI API key.")
if not os.getenv("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME"):
    raise ValueError("AZURE_OPENAI_RESPONSES_DEPLOYMENT_NAME environment variable is not set. Please set it to your Azure OpenAI responses deployment name.")




client = AzureResponsesAgent.create_client()

# 2. Create a Semantic Kernel agent for the OpenAI Responses API
agent = AzureResponsesAgent(
    ai_model_id="gpt-4o-mini",
    client=client,
    instructions="Answer questions about the menu.",
    name="Host",
    plugins=[MenuPlugin()],
)

# 3. Create a thread for the agent
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: ResponsesAgentThread = None

########################################
# invoke mode 
########################################
# for user_input in USER_INPUTS:
#     print(f"# User: '{user_input}'")
#     # 4. Invoke the agent for the current message and print the response
#     response = await agent.get_response(messages=user_input, thread=thread)
#     print(f"# {response.name}: {response.content}")
#     thread = response.thread

########################################
# invoke_stream mode 
########################################
for user_input in USER_INPUTS:
        print(f"# User: '{user_input}'")
        first_chunk = True
        # 4. Invoke the agent for the current message and print the response
        async for response in agent.invoke_stream(messages=user_input, thread=thread):
            thread = response.thread
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(response.content, end="", flush=True)
        print()

await thread.delete() if thread else None

# User: 'Hello'


# Host: Hi there! How can I assist you today?
# User: 'What is the special soup?'
# Host: The special soup is Clam Chowder. Is there anything else you'd like to know?
# User: 'How much does that cost?'
# Host: The Clam Chowder costs $9.99. Would you like to know anything else?
# User: 'Thank you'
# Host: You're welcome! If you have any more questions, feel free to ask. Enjoy your day!


# Use tools to answer specific questions 

## 🧪 Case 3.1 code interpreter with Azure AI Agent 
---

In [32]:
TASK = "Use code to determine the values in the Fibonacci sequence that that are less then the value of 101."

ai_agent_settings = AzureAIAgentSettings()

async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(credential=creds) as client,
):
    # 1. Create an agent with a code interpreter on the Azure AI agent service
    code_interpreter = CodeInterpreterTool()
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )

    # 2. Create a Semantic Kernel agent for the Azure AI agent
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
    )

    # 3. Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread = None

    try:
        print(f"# User: '{TASK}'")
        # 4. Invoke the agent for the specified thread for response
        async for response in agent.invoke(messages=TASK, thread=thread):
            if response.role != AuthorRole.TOOL:
                print(f"# Agent: {response}")
            thread = response.thread
    finally:
        # 6. Cleanup: Delete the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

The create method is deprecated. Use the __new__ method instead.


# User: 'Use code to determine the values in the Fibonacci sequence that that are less then the value of 101.'
# Agent: # Create a Fibonacci sequence with numbers less than 101
fibonacci_sequence = []
a, b = 0, 1  # Starting values of Fibonacci numbers

while a < 101:
    fibonacci_sequence.append(a)
    a, b = b, a + b  # Move to the next Fibonacci number

fibonacci_sequence
# Agent: The values in the Fibonacci sequence that are less than 101 are:

\[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89\]


## 🧪 Case 3.2 open API with Azure AI Agent 
---

In [44]:
USER_INPUTS = [
    "What is the name and population of the country that uses currency with abbreviation KRW",
    "What is the current weather in the capital city of the country?",
]

ai_agent_settings = AzureAIAgentSettings()

async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(credential=creds) as client,
):
    # 1. Read in the OpenAPI spec files
    openapi_spec_file_path = "resources"
    with open(os.path.join(openapi_spec_file_path, "weather.json")) as weather_file:
        weather_openapi_spec = json.loads(weather_file.read())
    with open(os.path.join(openapi_spec_file_path, "countries.json")) as countries_file:
        countries_openapi_spec = json.loads(countries_file.read())

    # 2. Create OpenAPI tools
    # Note that connection or managed identity auth setup requires additional setup in Azure
    auth = OpenApiAnonymousAuthDetails()
    openapi_weather = OpenApiTool(
        name="get_weather",
        spec=weather_openapi_spec,
        description="Retrieve weather information for a location",
        auth=auth,
    )
    openapi_countries = OpenApiTool(
        name="get_country",
        spec=countries_openapi_spec,
        description="Retrieve country information",
        auth=auth,
    )

    # 3. Create an agent on the Azure AI agent service with the OpenAPI tools
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        tools=openapi_weather.definitions + openapi_countries.definitions,
    )

    # 4. Create a Semantic Kernel agent for the Azure AI agent
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
    )

    # 5. Create a thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread = None

    try:
        for user_input in USER_INPUTS:
            print(f"# User: '{user_input}'")
            # 7. Invoke the agent for the specified thread for response
            async for response in agent.invoke(messages=user_input, thread=thread):
                if response.role != AuthorRole.TOOL:
                    print(f"# Agent: {response}")
                thread = response.thread
    finally:
        # 8. Cleanup: Delete the thread and agent
        await client.agents.delete_agent(agent.id)



# User: 'What is the name and population of the country that uses currency with abbreviation KRW'
# Agent: I encountered an issue retrieving the information for the currency "KRW." However, I can provide the details directly:

The currency abbreviation "KRW" stands for the South Korean Won, which is used in **South Korea**. The population of South Korea is approximately **51.8 million** as of 2023.
# User: 'What is the current weather in the capital city of the country?'
# Agent: The current weather in **Seoul**, South Korea (capital city), is:

- **Temperature**: 28°C (83°F)  
- **Feels Like**: 29°C (85°F)  
- **Condition**: Partly cloudy  
- **Humidity**: 58%  
- **Wind Speed**: 9 km/h (6 mph) from the WSW  
- **Visibility**: 10 km  

Do you need a detailed forecast or any specific weather details?


## 🧪 Case 3.3 bing search API as plugin with ChatCompletionAgent (optional, deprecated) 
---

In [52]:
# Simulate a conversation with the agent
USER_INPUTS = [
    "What is today's weather in South Korea?"
]
from semantic_kernel.connectors.search.bing import BingSearch
from semantic_kernel.functions import KernelArguments, KernelParameterMetadata, KernelPlugin

webplugin = KernelPlugin.from_text_search_with_search(
        BingSearch(api_key=os.getenv("BING_API_KEY")),
        plugin_name="bing",
        description="Search the web for information.",
        parameters=[
            KernelParameterMetadata(
                name="query",
                description="The search query.",
                type="str",
                is_required=True,
                type_object=str,
            ),
            KernelParameterMetadata(
                name="top",
                description="The number of results to return.",
                type="int",
                is_required=False,
                default_value=1,
                type_object=int,
            ),
            KernelParameterMetadata(
                name="skip",
                description="The number of results to skip.",
                type="int",
                is_required=False,
                default_value=0,
                type_object=int,
            ),
            # KernelParameterMetadata(
            #     name="site",
            #     description="The site to search.",
            #     default_value="https://github.com/microsoft/semantic-kernel/tree/main/python",
            #     type="str",
            #     is_required=False,
            #     type_object=str,
            # ),
        ],
    )



agent = ChatCompletionAgent(
        service=AzureChatCompletion(),
        name="Host",
        instructions="Answer questions from web search results. Add the web search reference url to the answer.",
        plugins=[webplugin],
    )

# 2. Create a thread to hold the conversation
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: ChatHistoryAgentThread = None

for user_input in USER_INPUTS:
    print(f"# User: {user_input}")
    # 4. Invoke the agent for a response
    response = await agent.get_response(messages=user_input, thread=thread)
    print(f"# {response.name}: {response} ")
    thread = response.thread


########################################
# invoke_stream mode 
########################################
# for user_input in USER_INPUTS:
#         print(f"# User: '{user_input}'")
#         first_chunk = True
#         # 4. Invoke the agent for the current message and print the response
#         async for response in agent.invoke_stream(messages=user_input, thread=thread):
#             thread = response.thread
#             if first_chunk:
#                 print(f"# {response.name}: ", end="", flush=True)
#                 first_chunk = False
#             print(response.content, end="", flush=True)
#         print()

await thread.delete() if thread else None

# User: What is today's weather in South Korea?


# Host: As of today, October 26, 2023, the weather in South Korea, particularly in Seoul, is generally clear and sunny. Current temperatures are around 19°C (66°F) with light winds. The conditions are favorable for outdoor activities, but it might get cooler in the evenings.

For more details, you can check the full weather report here: [Weather Report for Seoul](https://www.weather.com/news/weather/news) (Please note that this URL is a placeholder, as specific URLs were not returned in the search). 


## 🧪 Case 3.4 file search tool with AzureAssistantAgent (Open AI Asisstant)
---

In [48]:

# Simulate a conversation with the agent
USER_INPUTS = {
    "Who is the youngest employee?",
    "Who works in sales?",
    "I have a customer request, who can help me?",
}

ai_agent_settings = AzureAIAgentSettings()

# 1. Create the client using Azure OpenAI resources and configuration
client = AzureAssistantAgent.create_client()

# 2. Read and upload the file to the Azure OpenAI assistant service
pdf_file_path = "resources/employees.pdf"


with open(pdf_file_path, "rb") as file:
    file = await client.files.create(file=file, purpose="assistants")

vector_store = await client.vector_stores.create(
    name="step4_assistant_file_search",
    file_ids=[file.id],
)

# 3. Create file search tool with uploaded resources
file_search_tool, file_search_tool_resources = AzureAssistantAgent.configure_file_search_tool(vector_store.id)

# 4. Create the assistant on the Azure OpenAI service with the file search tool
definition = await client.beta.assistants.create(
    model=ai_agent_settings.model_deployment_name,
    instructions="Find answers to the user's questions in the provided file.",
    name="FileSearch",
    tools=file_search_tool,
    tool_resources=file_search_tool_resources,
)

# 5. Create a Semantic Kernel agent for the Azure OpenAI assistant
agent = AzureAssistantAgent(
    client=client,
    definition=definition,
)

# 6. Create a new thread for use with the assistant
# If no thread is provided, a new thread will be
# created and returned with the initial response
thread: AssistantAgentThread = None

try:
    for user_input in USER_INPUTS:
        print(f"# User: '{user_input}'")
        # 7. Invoke the agent for the current thread and print the response
        async for response in agent.invoke(messages=user_input, thread=thread):
            print(f"# Agent: {response}")
            thread = response.thread
finally:
    # 9. Clean up the resources
    await client.files.delete(file.id)
    await client.vector_stores.delete(vector_store.id)
    await client.beta.threads.delete(thread.id)
    await client.beta.assistants.delete(agent.id)

# User: 'Who works in sales?'
# Agent: Based on the provided file, the individuals working in sales are:

1. Mariam Jaslyn - Sales Representative【4:0†employees.pdf】.
2. Hicran Bea - Sales Manager【4:0†employees.pdf】.
3. Angelino Embla - Sales Representative【4:0†employees.pdf】.
# User: 'I have a customer request, who can help me?'
# Agent: To assist with customer requests, you may consider reaching out to the sales team:

1. **Mariam Jaslyn** - Sales Representative【4:0†employees.pdf】.
2. **Hicran Bea** - Sales Manager【4:0†employees.pdf】.
3. **Angelino Embla** - Sales Representative【4:0†employees.pdf】.

Additionally, the **Marketing Specialist** or **Administrative Assistant** roles may help depending on the nature of the request.
# User: 'Who is the youngest employee?'
# Agent: The youngest employee is **Teodor Britton**, born on January 9, 1997【12:0†employees.pdf】.


## 🧪 Case 3.5 BingGrounding with Azure AI Agent (SK)
---

In [68]:
from azure.ai.agents.models import BingGroundingTool, MessageRole
from azure.identity.aio import DefaultAzureCredential

from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread
from semantic_kernel.contents import (
    AnnotationContent,
    ChatMessageContent,
    FunctionCallContent,
    FunctionResultContent,
)
from langchain.prompts import PromptTemplate
import os
import pytz
from datetime import datetime
from IPython.display import Markdown, display



BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")
# BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_CONNECTION_NAME = os.getenv("BING_GROUNDING_CONNECTION_NAME")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 3))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")
    
ai_agent_settings = AzureAIAgentSettings()

async def handle_intermediate_steps(message: ChatMessageContent) -> None:
    for item in message.items or []:
        if isinstance(item, FunctionResultContent):
            print(f"Function Result:> {item.result} for function: {item.name}")
        elif isinstance(item, FunctionCallContent):
            print(f"Function Call:> {item.name} with arguments: {item.arguments}")
        else:
            print(f"{item}")

async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(credential=creds) as client,
    ):
        # 1. Enter your Bing Grounding Connection Name
        bing_connection = await client.connections.get(name=BING_GROUNDING_CONNECTION_NAME)
        conn_id = bing_connection.id

        # 2. Initialize agent bing tool and add the connection id
        bing = BingGroundingTool(connection_id=conn_id, market=BING_GROUNDING_MARKET, set_lang=BING_GROUNDING_SET_LANG, count=int(BING_GROUNDING_MAX_RESULTS))

        # 3. Create an agent with Bing grounding on the Azure AI agent service
        agent_definition = await client.agents.create_agent(
            name="SKBingGroundingAgent",
            instructions="Use the Bing grounding tool to answer the user's question.",
            model=AzureAIAgentSettings().model_deployment_name,
            tools=bing.definitions,
        )

        # 4. Create a Semantic Kernel agent for the Azure AI agent
        agent = AzureAIAgent(
            client=client,
            definition=agent_definition,
        )

        # 5. Create a thread for the agent
        # If no thread is provided, a new thread will be
        # created and returned with the initial response
        thread: AzureAIAgentThread | None = None
        
        SEARCH_GEN_USER_PROMPT_TEMPLATE = """
            please provide as rich and specific an answer and reference links as possible for `{query}`.
            Today is {current_date}. Results should be based on the recent information available.
        """
    
        SEARCH_GEN_USER_PROMPT = PromptTemplate(
            template=SEARCH_GEN_USER_PROMPT_TEMPLATE,
            input_variables=["query","current_date"]
        )
        
        current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")

        try:
            USER_INPUT = "가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?"
            print(f"# User: '{USER_INPUT}'")
            # 6. Invoke the agent for the specified thread for response
            async for response in agent.invoke(
                messages=SEARCH_GEN_USER_PROMPT.format(
                    query=USER_INPUT,
                    current_date=current_date
                ), thread=thread, on_intermediate_message=handle_intermediate_steps
            ):
                print(f"# {response.name}:")
                display(Markdown(response.message.content))
                thread = response.thread

                # 7. Show annotations
                if any(isinstance(item, AnnotationContent) for item in response.items):
                    for annotation in response.items:
                        if isinstance(annotation, AnnotationContent):
                            print(
                                f"Annotation :> {annotation.url}, source={annotation.quote}, with "
                                f"start_index={annotation.start_index} and end_index={annotation.end_index}"
                            )
        finally:
            # 8. Cleanup: Delete the thread and agent
            await thread.delete() if thread else None
            await client.agents.delete_agent(agent.id)



# User: '가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?'
Function Call:> bing_grounding with arguments: {'requesturl': 'https://api.bing.microsoft.com/v7.0/search?q=가장 최근에 서울 도심에 새로 오픈한 핫플레이스 2025년 6월', 'response_metadata': "{'market': 'ko-KR', 'num_docs_retrieved': 3, 'num_docs_actually_used': 3}"}
# SKBingGroundingAgent:


2025년 6월 현재 서울 도심에서 주목받는 새로운 핫플레이스 몇 곳을 소개합니다:

1. **노량진수산시장 현대화단지**: 기존 수산시장을 완전히 현대화하여 깔끔하고 쾌적한 환경에서 신선한 해산물을 직접 고르고 식사할 수 있는 장소입니다【3:0†source】.

2. **홍대 미디어 아트 갤러리**: 미디어 아트를 중심으로 한 전시와 공연 공간이 새롭게 오픈하여 예술적 영감을 주는 장소로 주목받고 있습니다【3:0†source】.

3. **여의도 한강공원 루프탑 카페**: 한강의 멋진 야경을 즐기며 음료와 디저트를 맛볼 수 있는 루프탑 공간이 새로운 명소로 떠올랐습니다【3:0†source】.

이 외에도 성수동의 다양한 갤러리형 카페, 잠실 석촌호수 근처의 뷰 카페 등이 계속해서 트렌드의 중심에 있으며, 한남동이나 익선동과 같은 문화적 개성의 동네도 꾸준히 인기를 끌고 있습니다【3:1†source】.

Annotation :> https://peppercorntc.com/2025%EB%85%84-%EC%84%9C%EC%9A%B8%ED%95%AB%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4-10%EA%B3%B3-%EB%86%93%EC%B9%98%EB%A9%B4-%ED%9B%84%ED%9A%8C/, source=【3:0†source】, with start_index=133 and end_index=145
Annotation :> https://peppercorntc.com/2025%EB%85%84-%EC%84%9C%EC%9A%B8%ED%95%AB%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4-10%EA%B3%B3-%EB%86%93%EC%B9%98%EB%A9%B4-%ED%9B%84%ED%9A%8C/, source=【3:0†source】, with start_index=229 and end_index=241
Annotation :> https://peppercorntc.com/2025%EB%85%84-%EC%84%9C%EC%9A%B8%ED%95%AB%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4-10%EA%B3%B3-%EB%86%93%EC%B9%98%EB%A9%B4-%ED%9B%84%ED%9A%8C/, source=【3:0†source】, with start_index=322 and end_index=334
Annotation :> https://presentlive.tistory.com/entry/%F0%9F%93%8C-2025%EB%85%84-%EC%84%9C%EC%9A%B8-%ED%95%AB%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4-TOP-5-%E2%80%93-%EC%9A%94%EC%A6%98-%EA%B0%80%EC%9E%A5-%ED%95%AB%ED%95%9C-%EA%B3%B3, source=【3:1†source】, with start_index=441 and end_index=45

## 🧪 Case 3.6 BingGrounding with Azure AI Agent (Vanila)
---

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import BingGroundingTool, MessageRole
from IPython.display import Markdown, display

# Create an Azure AI Client from an endpoint, copied from your Azure AI Foundry project.
# You need to login to Azure subscription via Azure CLI and set the environment variables
BING_GROUNDING_PROJECT_ENDPOINT = os.getenv("BING_GROUNDING_PROJECT_ENDPOINT")

BING_GROUNDING_CONNECTION_ID = os.getenv("BING_GROUNDING_CONNECTION_ID")
BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME")
BING_GROUNDING_MAX_RESULTS = int(os.getenv("BING_GROUNDING_MAX_RESULTS", 3))
BING_GROUNDING_MARKET = os.getenv("BING_GROUNDING_MARKET", "ko-KR")
BING_GROUNDING_SET_LANG = os.getenv("BING_GROUNDING_SET_LANG", "ko-KR")


# Create an AIProjectClient instance
project_client = AIProjectClient(
    endpoint=BING_GROUNDING_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),  # Use Azure Default Credential for authentication
)

# Initialize the Bing Grounding tool
bing = BingGroundingTool(connection_id=BING_GROUNDING_CONNECTION_ID)

with project_client:
    # Create an agent with the Bing Grounding tool
    agent = project_client.agents.create_agent(
        model=BING_GROUNDING_AGENT_MODEL_DEPLOYMENT_NAME,  # Model deployment name
        name="VanilaBingGroundingAgent",  # Name of the agent
        instructions="You are a helpful agent",  # Instructions for the agent
        tools=bing.definitions,  # Attach the Bing Grounding tool
    )
    print(f"Created agent, ID: {agent.id}")
    
    # Create a thread for communication
    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")


    SEARCH_GEN_USER_PROMPT_TEMPLATE = """
        please provide as rich and specific an answer and reference links as possible for `{query}`.
        Today is {current_date}. Results should be based on the recent information available.
    """

    SEARCH_GEN_USER_PROMPT = PromptTemplate(
        template=SEARCH_GEN_USER_PROMPT_TEMPLATE,
        input_variables=["query","current_date"]
    )

    current_date = datetime.now(tz=pytz.timezone("Asia/Seoul")).strftime("%Y-%m-%d")

    USER_INPUT = "가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?"
    print(f"# User: '{USER_INPUT}'")

    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role=MessageRole.USER,  # Role of the message sender
        content=SEARCH_GEN_USER_PROMPT.format(
            query=USER_INPUT,
            current_date=current_date
        )
    )
    print(f"Created message, ID: {message['id']}")

    # Create and process an agent run
    run = project_client.agents.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    # Check if the run failed
    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Fetch and log all messages
    message = project_client.agents.messages.get_last_message_by_role(thread_id=thread.id, role=MessageRole.AGENT)
    
    print(f"Role: {message.role}")
    # Extract the text content from the message
    if isinstance(message.content, list):
        for content_item in message.content:
            if content_item.get('type') == 'text':
                display(Markdown(content_item['text']['value']))
    else:
        display(Markdown(message.content))

    # Delete the agent when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Created agent, ID: asst_Ig5WOsYOCGGu5iHH8dCRRm8Y
Created thread, ID: thread_EiknvevrOdXAv0qmDmXmxA0l
# User: '가장 최근에 서울 도심에 새로 오픈한 핫플레이스는?'
Created message, ID: msg_BBtZhrsSeBpunqhdhRDU49ZA
Run finished with status: RunStatus.COMPLETED
Role: MessageRole.AGENT


가장 최근에 서울 도심에 새로 오픈한 핫플레이스와 관련해 2025년 현재 서울에서 주목받는 여러 핫플레이스를 소개하겠습니다.

1. 성수동:  
- '오르에르'라는 베이글 전문 카페가 최근 인기를 끌고 있습니다. 성수동은 공장지대에서 감각적인 카페와 갤러리가 밀집한 트렌디한 지역으로 변모했으며, '오르에르'는 아침 일찍 오픈해 쫀득한 베이글과 크림치즈 등이 인기 메뉴입니다.

2. 한남동:  
- '어니언'이라는 카페 겸 브런치 맛집이 명성을 얻고 있습니다. 빈티지한 분위기의 오래된 주택을 개조해 아보카도 토스트와 바질 페스토 파스타 등이 주력 메뉴입니다.

3. 연남동:  
- 일식과 퓨전을 접목한 '연남토마'가 최근 입소문을 타고 있습니다. 명란 크림 우동과 연어 덮밥 등이 인기이며, 일본식 선술집 분위기가 특징입니다.

4. 이태원:  
- 비건 디저트 전문 카페 '플랜트'가 건강과 환경을 생각하는 트렌드에 맞춰 각광받고 있습니다. 코코넛 초콜릿 케이크와 당근 케이크 등이 대표 메뉴입니다.

5. 여의도:  
- 독일식 빵과 브런치 카페 '더베이커스테이블'이 최근 주목받고 있습니다. 호밀빵 샌드위치와 프레첼 등이 인기 메뉴이며, 한강공원 근처여서 식사 후 산책하기 좋습니다.

서울 도심의 새로운 핫플레이스들은 각 지역 특성을 잘 반영하여 지속 가능성, 개성, 건강 트렌드 등을 반영한 공간들이 많습니다. 특히 성수동과 한남동, 연남동, 이태원, 여의도 지역이 2025년에도 활발히 새 명소들이 생겨나고 있으며, 각각 독특한 분위기와 맛집을 갖추고 있습니다.

더 자세한 내용과 위치, 메뉴, 운영시간 등은 아래 링크에서 확인할 수 있습니다.  
- 2025년 서울 핫플레이스 추천 및 상세 가이드: https://ifif2y.tistory.com/entry/2025%EB%85%84-%EC%84%9C%EC%9A%B8-%ED%95%AB%ED%94%8C%EB%A0%88%EC%9D%B4%EC%8A%A4-%EC%B6%94%EC%B2%9C-%ED%8A%B8%EB%A0%8C%EB%94%94%ED%95%9C-%EB%AA%85%EC%86%8C%EC%99%80-%EC%88%A8%EC%9D%80-%EB%B3%B4%EC%84%9D  
- 서울 핫플레이스 TOP 5 및 가볼 만한 곳: https://blog.naver.com/PostView.naver?blogId=trendreviewer&logNo=223719780932  

요약하자면, 2025년 현재 성수동의 '오르에르', 한남동 '어니언', 연남동 '연남토마', 이태원의 '플랜트', 여의도의 '더베이커스테이블' 등이 서울 도심에서 새로 오픈해 인기 있는 핫플레이스로 꼽힙니다.

Deleted agent
